In [40]:
import pandas as pd
import os
from Bio import SeqIO

In [ ]:
# get list of all Mnat genes
mnat = SeqIO.parse(handle='../../TR_20230707_orthofinder/input/Mnat_gene_v1.2.pep.fa', format='fasta')
seqID = [i.name for i in mnat]

In [55]:
Mnat_to_Mbel_orthologues = pd.read_csv('../../TR_20230707_orthofinder/input/OrthoFinder/Results_Jul12/Orthologues/Orthologues_Mnat_gene_v1.2.pep/Mnat_gene_v1.2.pep__v__Mbel.aa.tsv', sep='\t')

In [84]:
[i.rsplit('.')[0] for i in os.listdir(folder) if i.endswith('.csv')]

['king_upregulated_vs_queen',
 'majorworker_upregulated_vs_king',
 'majorworker_upregulated_vs_queen',
 'queen_upregulated_vs_king',
 'queen_upregulated_vs_majorworker',
 'king_upregulated_vs_majorworker',
 'minorworker_upregulated_vs_majorworker',
 'king_upregulated_vs_minorworker',
 'minorworker_upregulated_vs_king',
 'majorworker_upregulated_vs_minorworker',
 'queen_upregulated_vs_minorworker',
 'minorworker_upregulated_vs_queen']

# get differential expression data

In [85]:
folder = "./DEG_Elsner2018/"
DEdict = {}

#get list of all genes
for ID in seqID:
    DEdict.setdefault(ID, {})
    DEdict[ID]['DE']=False
    DEdict[ID]['DE_royal']=False
    DEdict[ID]['sgo']=False
    DEdict[ID]['sgo_mbel']=False
    DEdict[ID]['sgo_mnat']=False

    DEdict[ID]['king_upregulated_vs_queen']=False
    DEdict[ID]['majorworker_upregulated_vs_king']=False
    DEdict[ID]['majorworker_upregulated_vs_queen']=False
    DEdict[ID]['queen_upregulated_vs_king']=False
    DEdict[ID]['queen_upregulated_vs_majorworker']=False
    DEdict[ID]['king_upregulated_vs_majorworker']=False
    DEdict[ID]['minorworker_upregulated_vs_majorworker']=False
    DEdict[ID]['king_upregulated_vs_minorworker']=False
    DEdict[ID]['minorworker_upregulated_vs_king']=False
    DEdict[ID]['majorworker_upregulated_vs_minorworker']=False
    DEdict[ID]['queen_upregulated_vs_minorworker']=False
    DEdict[ID]['minorworker_upregulated_vs_queen']=False

# get differential expression data
for file in [i for i in os.listdir(folder) if i.endswith('.csv')]:
    path = os.path.join(folder, file)
    df = pd.read_csv(path, index_col=0)
    for i, k in df.iterrows():
        DEdict.setdefault(k['ID'], {})[file.rsplit('.')[0]]=True             #.setdefault('de_type',[]).append(file.rsplit('.')[0])
        DEdict[k['ID']]['DE']=True
        flist = file.rsplit('.')[0].split('_')
        if flist[0] in ['king', 'queen']:
            if flist[3] in ['majorworker', 'minorworker']:
                DEdict[k['ID']]['DE_royal']=True
        elif flist[0] in ['majorworker', 'minorworker']:
            if flist[3] in ['king', 'queen']:
                DEdict[k['ID']]['DE_royal']=True

# get information on orthologues
for i, k in Mnat_to_Mbel_orthologues.iterrows():
    Mnats = [i.strip(' \t') for i in  k['Mnat_gene_v1.2.pep'].split(',')]
    Mbels = [i.strip(' \t') for i in k['Mbel.aa'].split(',')]
    if len(Mnats) == 1:
        if len(Mbels) == 1:
            DEdict[Mnats[0]]['sgo']=True
            DEdict[Mnats[0]]['sgo_mnat']=True
            DEdict[Mnats[0]]['sgo_mbel']=True
        else:
            DEdict[Mnats[0]]['sgo_mnat']=True
    else:    
        if len(Mbels) == 1:
            DEdict[Mnats[0]]['sgo_mbel']=True
    
    for i in Mnats:
        DEdict[i]['orthologues']=Mbels

In [87]:
#DEdict

# get Mnat and Mbel Orthologues

True

In [109]:
DE_table = []

for GeneID, item in DEdict.items():
    #for key, item2 in item.items():
    line = [GeneID]
    for k,i in item.items():
        if isinstance(i,str)==True:
            line.append(i)
        elif isinstance(i, bool):
            line.append(i)
        else:
            line.append(';'.join(i))

    DE_table.append(line)
DEdf = pd.DataFrame(DE_table)

In [104]:
DEdict['Mnat_01286'].keys()

dict_keys(['DE', 'DE_royal', 'sgo', 'sgo_mbel', 'sgo_mnat', 'king_upregulated_vs_queen', 'majorworker_upregulated_vs_king', 'majorworker_upregulated_vs_queen', 'queen_upregulated_vs_king', 'queen_upregulated_vs_majorworker', 'king_upregulated_vs_majorworker', 'minorworker_upregulated_vs_majorworker', 'king_upregulated_vs_minorworker', 'minorworker_upregulated_vs_king', 'majorworker_upregulated_vs_minorworker', 'queen_upregulated_vs_minorworker', 'minorworker_upregulated_vs_queen', 'orthologues'])

In [110]:
DEdf.columns = ['GeneID','DE', 'DE_royal', 'sgo', 'sgo_mbel', 'sgo_mnat', 'king_upregulated_vs_queen', 'majorworker_upregulated_vs_king', 'majorworker_upregulated_vs_queen', 'queen_upregulated_vs_king', 'queen_upregulated_vs_majorworker', 'king_upregulated_vs_majorworker', 'minorworker_upregulated_vs_majorworker', 'king_upregulated_vs_minorworker', 'minorworker_upregulated_vs_king', 'majorworker_upregulated_vs_minorworker', 'queen_upregulated_vs_minorworker', 'minorworker_upregulated_vs_queen', 'orthologues']

In [118]:
DEdf.to_csv('./20230727_DE_table_with_orthologues_from_elsner2017.tsv', sep='\t')